<a href="https://colab.research.google.com/github/juhumkwon/Defense_Cloud/blob/main/9_0_2_word_embedding_positional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np

# 1. 문장 처리 (토큰화, 임베딩, 포지셔널 인코딩 추가)

# 예시 문장
tokens = ["나는", "학교에", "간다"]

# 어휘 크기와 임베딩 차원 정의
vocab_size = 10000  # 어휘 크기
embedding_dim = 512  # 임베딩 차원

# 예시 토큰 ID (가정)
token_ids = [42, 23, 77]
token_ids = tf.constant(token_ids)
print("token_ids:", token_ids)
print()

# 임베딩 레이어 생성
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# 토큰 ID → 임베딩 벡터
# 여기서 embedded_tokens는 문장의 각 단어를 임베딩 벡터로 표현한 결과입니다.
embedded_tokens = embedding_layer(token_ids)  # shape: (3, 512)
print("embedded_tokens:", embedded_tokens)
print()

# 포지셔널 인코딩 생성 함수
def get_positional_encoding(seq_len, embedding_dim):
    # 각 위치 (seq_len,) -> (seq_len, 1)
    positions = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
    # 임베딩 차원 인덱스 (embedding_dim,) -> (1, embedding_dim)
    i = tf.range(embedding_dim, dtype=tf.float32)[tf.newaxis, :]

    # angle rate 계산 (1 / 10000^(2i/d_model))
    angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(embedding_dim, tf.float32))
    angles = positions * angle_rates  # shape: (seq_len, embedding_dim)

    # 짝수 인덱스는 sin, 홀수 인덱스는 cos
    pos_encoding = tf.where(
        tf.cast(i, tf.int32) % 2 == 0,
        tf.sin(angles),
        tf.cos(angles)
    )

    print("pos_encoding_shape:", pos_encoding)
    print()

    return pos_encoding  # shape: (seq_len, embedding_dim)


# 포지셔널 인코딩 추가
seq_len = len(tokens)
positional_encoding = get_positional_encoding(seq_len, embedding_dim)
input_with_positional_encoding = embedded_tokens + positional_encoding

print("임베딩 벡터 (단어 의미 + 포지셔널 인코딩):")
print("input_with_positional_encoding:", input_with_positional_encoding.numpy())
print()

# 2. 자기-어텐션 (Self-Attention) 계산
# Query, Key, Value 생성
query = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)
key = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)
value = tf.keras.layers.Dense(embedding_dim)(input_with_positional_encoding)  # shape: (3, 512)

print("Query shape:", query.shape)
print("Key shape:", key.shape)
print("Value shape:", value.shape)

# 어텐션 스코어 계산 (Q * K^T)
attention_scores = tf.matmul(query, key, transpose_b=True)  # shape: (3, 3)
print("attention_scores:", attention_scores.shape)
print()

# 소프트맥스를 통해 가중치 계산
attention_weights = tf.nn.softmax(attention_scores, axis=-1)  # shape: (3, 3)
print("attention_weights:", attention_weights)
print()

# 어텐션 출력 (Attention Weight * Value)
attention_output = tf.matmul(attention_weights, value)  # shape: (3, 512)

# 3. 결과 출력

print("\n어텐션 후 출력 벡터:")
print(attention_output.numpy())


token_ids: tf.Tensor([42 23 77], shape=(3,), dtype=int32)

embedded_tokens: tf.Tensor(
[[ 0.04419316 -0.01610865  0.02883747 ...  0.01627636  0.00515466
  -0.00834884]
 [-0.02877828 -0.01684544 -0.03997324 ... -0.00483409 -0.01849253
   0.02204117]
 [-0.04094719 -0.03268226 -0.00014751 ...  0.04200516  0.04473258
  -0.04679233]], shape=(3, 512), dtype=float32)

pos_encoding_shape: tf.Tensor(
[[ 0.0000000e+00  1.0000000e+00  0.0000000e+00 ...  1.0000000e+00
   0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  5.4030228e-01  8.2185626e-01 ...  1.0000000e+00
   1.0366329e-04  1.0000000e+00]
 [ 9.0929741e-01 -4.1614681e-01  9.3641472e-01 ...  1.0000000e+00
   2.0732658e-04  1.0000000e+00]], shape=(3, 512), dtype=float32)

임베딩 벡터 (단어 의미 + 포지셔널 인코딩):
input_with_positional_encoding: [[ 0.04419316  0.98389137  0.02883747 ...  1.0162764   0.00515466
   0.9916512 ]
 [ 0.8126927   0.5234568   0.781883   ...  0.9951659  -0.01838887
   1.0220412 ]
 [ 0.8683502  -0.44882908  0.9362672  ...  1.0420052 